# 1 The ProShares ETF Product

### 1. “Alternative ETFs”
Describe the two types of investments referenced by this term.
- alternative asset classes (giving
exposure to nonmainstream assets) and
- alternative strategies (pursuing investment
processes that were not constrained and could
use leverage, shorting, and derivatives).

### 2. Hedge Funds.

#### (a) Using just the information in the case, what are two measures by which hedge funds are an attractive investment?
- 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## 2 Analyzing the Data

#### Use the data found on Canvas, in ‘proshares analysis data.xlsx’. It has monthly data on financial indexes and ETFs from Aug 2011 through Sep 2021.

### 2.1 For the series in the “hedge fund series” tab, report the following summary statistics:1
- (a) Mean
- (b) Volatility
- (c) Sharpe ratio Annualize these statistics.